This notebook selects data from 100 TF dataset based on the list25_path argument. This points at a csv file that should contain a column named TF (with a list of TFs to choose). Note: I have manually added GPBP1L1 to the original file, creating top_25_summary_GPBP1L1.csv (to have same dataset as Amber).

In [1]:
import os
import shutil
import sys
import pandas as pd

In [2]:
samplefile_outdir = '/home/shush/profile/tfprofile/datasets/samplefiles/'
list25_path = 'datasets/top_25_summary.csv'
# list25_path = 'datasets/top_25_summary_GPBP1L1.csv'
list25_df = pd.read_csv(list25_path, index_col=0)
selected_tfs = list25_df['TFs'].values # make database from these

# 100TF dataset location
data_dir = '/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/100_TF'
summary_path = os.path.join(data_dir, 'summary.csv')
summary_df = pd.read_csv(summary_path, index_col=0)
labels = [l.split('TF-ChIP-seq_')[-1].split('-human')[0] for l in summary_df['label'].values]



In [3]:
rows = []
for tf in selected_tfs:
    if tf in labels:
        print('Adding TF: ', tf)
        i = labels.index(tf)
        rows.append(summary_df.iloc[i,:])

Adding TF:  CTCF
Adding TF:  MNT
Adding TF:  UBTF
Adding TF:  CREB1
Adding TF:  ZFY
Adding TF:  ARID5B
Adding TF:  EGR1
Adding TF:  ZNF574
Adding TF:  HNF4A
Adding TF:  ZNF776
Adding TF:  ZBTB14
Adding TF:  SP2
Adding TF:  ELF1
Adding TF:  FOXP4
Adding TF:  ZNF709
Adding TF:  ZNF547
Adding TF:  ZNF331
Adding TF:  TCF7L2
Adding TF:  HIVEP1
Adding TF:  ZNF274
Adding TF:  ZNF800
Adding TF:  LCORL
Adding TF:  RXRA
Adding TF:  USF1
Adding TF:  DDIT3


In [4]:
result_path = 'datasets/summary_25_links.csv'
summary_25 = pd.DataFrame(rows)
summary_25.to_csv(result_path)

In [5]:
for folder in ['raw', 'sign', 'fold', 'bed']:
    if folder == 'raw':
        filetype = 'bam'
    else:
        filetype = folder
    folder_dir = os.path.join(data_dir, folder)
    filenames = [file.split('/')[-1] for file in summary_25[filetype].values]
    if folder=='raw':
        filenames = [file.replace('bam', 'bw') for file in filenames]
    assert all([f in os.listdir(folder_dir) for f in filenames]), 'File not found'
    labels = summary_25['label'].values
    file_paths = [os.path.join(folder_dir, file) for file in filenames]

    if filetype=='bed':
        label = 'top25'
        with open(os.path.join(samplefile_outdir, 'basset_sample_beds_{}.txt'.format(label)), 'w') as filehandle:
            for i in range(len(labels)):
                filehandle.write('{}\t{}\n'.format(labels[i], file_paths[i]))
    else:
        identifiers = [filename.split('.')[0] for filename in filenames]

        first_line = '\t'.join(['index', 'identifier', 'file', 'sum_stat', 'description'])
        label = 'top25_'+filetype
        with open(os.path.join(samplefile_outdir, 'basenji_sample_{}.txt'.format(label)), 'w') as filehandle:
            filehandle.write('{}\n'.format(first_line))
            for i in range(len(file_paths)):
                filehandle.write('{}\t{}\t{}\t{}\t{}\n'.format(i, identifiers[i],
                                                               file_paths[i],
                                                               'sum', labels[i]))